# Lab 1: Langfuse Tracing

In this lab, we will learn how to use Langfuse tracing to log and analyze the execution of your LLM applications. The Langfuse supports self-hosted on AWS (this lab) and there is a [cloud version](https://cloud.langfuse.com/) available. [Tracing](https://langfuse.com/docs/tracing) in Langfuse is a way to log and analyze the execution of your LLM applications and following reference provides a detailed overview of the data model used. It is inspired by OpenTelemetry.


## [Traces and Observations](https://langfuse.com/docs/tracing-data-model)
A trace typically represents a single request or operation. It contains the overall input and output of the function, as well as metadata about the request, such as the user, the session, and tags. Usually, a trace corresponds to a single api call of an application.

Each trace can contain multiple observations to log the individual steps of the execution.

- Observations are of different types:
    - Events are the basic building blocks. They are used to track discrete events in a trace.
    - Spans represent durations of units of work in a trace.
    - Generations are spans used to log generations of AI models. They contain additional attributes about the model, the prompt, and the completion. For generations, [token usage and costs](https://langfuse.com/docs/model-usage-and-cost) are automatically calculated.
- Observations can be nested.

![Trace and Observations](./images/trace-observation.png)
![Trace and Observations UI](./images/trace-observation-ui.png)

## [Sessions](https://langfuse.com/docs/tracing-data-model)
Optionally, traces can be grouped into sessions. Sessions are used to group traces that are part of the same user interaction. A common example is a thread in a chat interface.
Please refer to the [Sessions documentation](https://langfuse.com/docs/sessions) to add sessions to your traces.

![Trace and Sessions](./images/trace-sessions.png)
![Trace and Sessions UI](./images/trace-sessions-ui.png)


## [Scores](https://langfuse.com/docs/tracing-data-model)

Traces and observations can be evaluated using [scores](https://langfuse.com/docs/scores/overview). Scores are flexible objects that store evaluation metrics and can be:

- Numeric, categorical, or boolean values
- Associated with a trace (required)
- Linked to a specific observation (optional)
- Annotated with comments for additional context
- Validated against a score configuration schema (optional)

![Trace and Scores](./images/trace-scores.png)

Please refer to the [scores documentation](https://langfuse.com/docs/scores/overview) to get started. For more details on score types and attributes, refer to the [score data model documentation](https://langfuse.com/docs/scores/data-model).




## Scores

Traces and observations can be evaluated using [scores](https://langfuse.com/docs/scores/overview). Scores are flexible objects that store evaluation metrics and can be:

- Numeric, categorical, or boolean values
- Associated with a trace (required)
- Linked to a specific observation (optional)
- Annotated with comments for additional context
- Validated against a score configuration schema (optional)

![Trace and Scores](./images/trace-scores.png)

[Source](https://langfuse.com/docs/scores/overview)

Please refer to the [scores documentation](https://langfuse.com/docs/scores/overview) to get started. For more details on score types and attributes, refer to the [score data model documentation](https://langfuse.com/docs/scores/data-model).


## Pre-requisites

> If you haven't selected the kernel, please click on the "Select Kernel" button at the upper right corner, select Python Environments and choose ".venv (Python 3.9.20) .venv/bin/python Recommended".

> To execute each notebook cell, press Shift + Enter.

> ℹ️ You can **skip these prerequisite steps** if you're in an instructor-led workshop using temporary accounts provided by AWS


### Dependencies and Environment Variables

We will use the langfuse and boto3:
- The Langfuse Python SDK along with the self-hosting deployment to debug and improve LLM applications by tracing model invocations, managing prompts / models configurations and running evaluations.
- The boto3 SDK to interact with models on Amazon Bedrock or Amazon SageMaker.

Run the following command to install the required Python SDKs.
> Please feel free to skip the installation if you are using a provided AWS account in a AWS organised event

In [42]:
# Uncomment the following line to install dependencies if you are not using AWS workshop environment
# %pip install -q langfuse boto3  --upgrade

Please make sure you have completed the prerequisites to setup the Langfuse project and API keys in the .env file to connect to self-hosted or cloud Langfuse environment.


In [2]:
# if you already define the environment variables in the .env of the vscode server, please skip the following cell
# Define the environment variables for langfuse
# You can find those values when you create the API key in Langfuse
# os.environ["LANGFUSE_SECRET_KEY"] = "xxxx" # Your Langfuse project secret key
# os.environ["LANGFUSE_PUBLIC_KEY"] = "xxxx" # Your Langfuse project public key
# os.environ["LANGFUSE_HOST"] = "xxx" # Langfuse domain

See [Langfuse documentation](https://langfuse.com/docs/get-started) for more details.

## Initialization and Authentication Check
Run the following cells to initialize common libraries and clients.

In [1]:
# import all the necessary packages
import sys
import os
from typing import Optional

import boto3
from langfuse import Langfuse
from langfuse.decorators import langfuse_context, observe
from langfuse.model import PromptClient

Let's create bedrock client and bedrock runtime client and please make sure the region is us-west-2 for this lab. The expected result is to see the following output:

```
Found Nova model: US Nova Pro - us.amazon.nova-pro-v1:0
Found Nova model: US Nova Lite - us.amazon.nova-lite-v1:0
Found Nova model: US Nova Micro - us.amazon.nova-micro-v1:0
```



> As Nova models in us-west-2 can only be called via Cross-Region Inference (CRIS), the model_id has "us." prefix to indicate this is a CRIS call. This can add latency to the model call.



In [ ]:
# used to access Bedrock configuration
# region has to be in us-west-2 for this lab
bedrock = boto3.client(service_name="bedrock", region_name="us-west-2")

# Check if Nova models are available in this region
models = bedrock.list_inference_profiles()
nova_found = False
for model in models["inferenceProfileSummaries"]:
    if (
        "Nova Pro" in model["inferenceProfileName"]
        or "Nova Lite" in model["inferenceProfileName"]
        or "Nova Micro" in model["inferenceProfileName"]
    ):
        print(
            f"Found Nova model: {model['inferenceProfileName']} - {model['inferenceProfileId']}"
        )
        nova_found = True
if not nova_found:
    raise ValueError(
        "No Nova models found in available models. Please ensure you have access to Nova models."
    )
#  Coverage, log level, etc.

Initialize the Langfuse client and check credentials are valid.

In [ ]:
from langfuse import Langfuse

# langfuse client
langfuse = Langfuse()
if langfuse.auth_check():
    print("Langfuse has been set up correctly")
    print(f"You can access your Langfuse instance at: {os.environ['LANGFUSE_HOST']}")
else:
    print(
        "Credentials not found or invalid. Check your Langfuse API key and host in the .env file."
    )

### Langfuse Wrappers for Bedrock Converse API 
You can use the Amazon Bedrock Converse API to create conversational applications that send and receive messages to and from an Amazon Bedrock model. For example, you can create a chat bot that maintains a conversation over many turns and uses a persona or tone customization that is unique to your needs, such as a helpful technical support assistant.

To use the Converse API, you use the Converse or ConverseStream (for streaming responses) operations to send messages to a model. It is possible to use the existing base inference operations (InvokeModel or InvokeModelWithResponseStream) for conversation applications. However, we recommend using the Converse API as it provides consistent API, that works with all Amazon Bedrock models that support messages. This means you can write code once and use it with different models. Should a model have unique inference parameters, the Converse API also allows you to pass those unique parameters in a model specific structure.

For more details, please refer to the [Carry out a conversation with the Converse API operations](https://docs.aws.amazon.com/bedrock/latest/userguide/conversation-inference.html).


In [4]:
sys.path.append(os.path.abspath('..'))  # Add parent directory to path
from config import GUARDRAIL_CONFIG, MODEL_CONFIG
from utils import converse, converse_tool_use

### Chat Examples

#### Define a helper function to call the  Converse API wrapper

> Please make sure your have setup the Nova custom model pricing per mentioned in  [Langfuse Setup](https://catalog.workshops.aws/genaiops-langfuse/en-US/00-introduction/langfuse-setup) under Introduction section of the workshop studio

In [5]:
@observe(name="Simple Chat")
def simple_chat(
    model_config: dict,
    messages: list,
    prompt: PromptClient = None,
    use_guardrails: bool = False,
) -> dict:
    """
    Executes a simple chat interaction using the specified model configuration.

    Args:
        model_config (dict): Configuration parameters for the chat model.
        messages (list): A list of message dictionaries to be processed.
        prompt (PromptClient, optional): Optional prompt client for advanced handling.
        use_guardrails (bool, optional): When True, applies additional guardrail configurations.

    Returns:
        dict: The response from the 'converse' function call.
    """
    config = model_config.copy()
    if use_guardrails:
        config["guardrailConfig"] = GUARDRAIL_CONFIG
    return converse(messages=messages, prompt=prompt, **config)

#### Use Case 1
let's start with a single turn chat use case and use Nova Pro as the default model.

In [ ]:
# Decorator to observe and track this function execution in Langfuse
@observe(name="Single Turn Example")
def chat_single_model(
    messages: list, model_type: str = "nova_pro", use_guardrails: bool = False
) -> dict:
    """
    Execute a single turn chat interaction using one specified Nova model.

    Args:
        messages (list): The user's input query
        model_type (str): The Nova model to use (nova_pro, nova_lite, or nova_micro)
        use_guardrails (bool): Whether to apply guardrails to the model invocation

    Returns:
        dict: Response containing model output and status code
    """
    langfuse_context.update_current_trace(
        user_id="nova-user-1",
        tags=["lab1", "single-turn"],
    )

    response = simple_chat(
        model_config=MODEL_CONFIG[model_type],
        messages=messages,
        use_guardrails=use_guardrails,
    )

    return {"model": model_type, "response": response, "statusCode": 200}


# Make a sample request to test the chat API
# Ask about luxury resort check-in process
print(
    chat_single_model(
        [
            {
                "role": "user",
                "content": "Explain the process of checking in a guest at a luxury resort, think step by step.",
            }
        ]
    )
)

# Force immediate sending of the trace data to Langfuse
# Rather than waiting for automatic flush
langfuse_context.flush()

In [ ]:
print(
    f"In Langfuse dashboard, you can find the summary of the traces, model costs and model usage.:\n{os.environ['LANGFUSE_HOST']}"
)

![Langfuse Dashboard](./images/langfuse-dashboard-use-case-1.png)


The detailed traced can be found in the **Traces** section and you can click on the trace to see the detailed trace. And there are some key insights in this trace.

- input token is 12 and output token is 662, in total there are 675 tokens used

- The model used is us.amazon.nova-pro-v1:0

- Model parameters are shown such as temperature, max_tokens, etc.

- Most importantly, the total cost of this invovation costs $0.002129 

![Langfuse Dashboard](./images/langfuse-trace-use-case-1.png)


#### Use Case 2
This use case demonstrates running a single trace within one session, where we'll execute three distinct observations using different Nova model variants for comparison.

In [ ]:
@observe(name="Multi-Turn Example")
def chat_compare_models(
    messages: list,
    model_types: list = ["nova_pro", "nova_lite", "nova_micro"],
    use_guardrails: bool = False,
) -> dict:
    """
    Execute the same query across all Nova models for comparison.

    Args:
        messages (list): The user's input query
        model_types (list): The Nova models to use (nova_pro, nova_lite, or nova_micro)
        use_guardrails (bool): Whether to apply guardrails to the model invocation
    Returns:
        dict: Responses from all models and status code
    """
    langfuse_context.update_current_trace(
        user_id="nova-user-1",
        session_id="model-comparison",
        tags=["lab1", "model-comparison"],
    )

    responses = {}
    for model_type in model_types:
        responses[model_type] = simple_chat(
            model_config=MODEL_CONFIG[model_type],
            messages=messages,
            use_guardrails=use_guardrails,
        )

    return {"responses": responses, "statusCode": 200}


# user request
print(
    chat_compare_models(
        [
            {
                "role": "user",
                "content": "Explain the process of checking in a guest at a luxury resort, think step by step.",
            }
        ]
    )
)

langfuse_context.flush()

So you can combine multiple observations into one trace and see the cost and usage of each observation. Nova micro has the lowest cost and fastest response time.

![langfuse-traces-use-case-2](./images/langfuse-trace-use-case-2.png)

#### Use Case 3
In this case, let's simulate a RAG use case with a dummy retrieval function called retrieve_context, it is a dummy function that returns a static context.
We will simply reuse the simple_chat function to chat with the model by passing the context from the retrieval function as part of the system prompt.

In [ ]:
CONTEXT = """1st January 2025
Sydney: 24 degrees celcius.
New York: 13 degrees celcius.
Tokyo: 11 degrees celcius."""


@observe(name="Dummy Retrieval")
def retrieve_context(query: str) -> str:
    """Retrieves static context for the given query."""
    return CONTEXT


@observe(name="RAG Example")
def rag_api(query: str) -> dict:
    """
    Performs a Retrieval-Augmented Generation (RAG) query using a static context.

    Args:
        query (str): The user query.

    Returns:
        dict: The response from the model and a status code.
    """
    langfuse_context.update_current_trace(
        user_id="nova-user-1",
        session_id="rag-session",
        tags=["lab1", "rag-example"],
    )

    context = retrieve_context(query)
    messages = [
        {
            "content": f"Context: {context}\nBased on the context above, answer the following question:",
            "role": "system",
        },
        {"content": query, "role": "user"},
    ]
    response = simple_chat(model_config=MODEL_CONFIG["nova_pro"], messages=messages)

    return {"response": response, "statusCode": 200}


# User request
print(rag_api("how you like the weather in Sydney? any comments?"))

langfuse_context.flush()

In the trace, you can see the retrieval function is called and the context is passed to the model as part of the system prompt. The fine model invocation takes both system prompt and user prompt and return the response.

![langfuse-traces-use-case-3](./images/langfuse-trace-use-case-3.png)


#### Use Case 4

Multi-Modal Capabilities with Image Support

Modern AI systems are increasingly adopting multi-modal capabilities, allowing them to process and understand different types of data inputs, including text, images, and audio. In this example, we demonstrate how Langfuse supports tracing for image-based inputs, which is particularly valuable for:

1. **Image Analysis**: Interpreting and describing visual content
2. **Visual Question Answering**: Answering questions based on image context
3. **Document Processing**: Extracting information from scanned documents or images
4. **Content Moderation**: Identifying inappropriate or sensitive visual content

The implementation uses a structured message format where the image URL is passed as part of the user prompt, enabling the model to process both textual queries and visual information simultaneously. This capability is especially useful in applications like:

- E-commerce product recognition
- Medical image analysis
- Social media content understanding

Langfuse's tracing capabilities extend to these multi-modal interactions, providing visibility into how the model processes and responds to image inputs, which is crucial for debugging and improving these complex systems.

In this use case, we will pass images as part of the user prompt and the model will process both the text and the image and langfuse will trace the entire process.

In [ ]:
@observe(name="Multi-Modal Image Example")
def vision_api(
    query: str,
    image_url: str,
) -> Optional[str]:
    langfuse_context.update_current_trace(
        user_id="nova-user-1",
        session_id="vision-session",
        tags=["lab1", "vision-example"],
    )

    messages = [
        {
            "role": "system",
            "content": "You are an AI trained to describe and interpret images.",
        },
        {
            "role": "user",
            "content": [
                {"type": "text", "text": query},
                {"type": "image_url", "image_url": {"url": image_url}},
            ],
        },
    ]

    return {
        "response": simple_chat(
            model_config=MODEL_CONFIG["nova_pro"], messages=messages
        ),
        "statusCode": 200,
    }


# image source: https://www.aboutamazon.com/news/aws/aws-reinvent-2024-keynote-live-news-updates
print(
    vision_api(
        query="What is happening in this image?",
        image_url="https://amazon-blogs-brightspot.s3.amazonaws.com/df/82/368cb270402e9739f04905ea9b19/swami-bedrock.jpeg",
    )
)

langfuse_context.flush()

Langfuse also supports tracing with image input, this is very useful for a multi-modal use case in which the model can take image as input.

![langfuse-traces-use-case-4](./images/langfuse-trace-use-case-4.png)


## Tool Use with Langfuse Tracing
Tool use enables AI models to interact with external functions and APIs, extending their capabilities beyond pure text generation. This is particularly useful for:
- Accessing real-time data (e.g., weather, stock prices)
- Performing complex calculations
- Integrating with external systems
- Extracting structured data from unstructured sources (e.g. text, images)

### Use Case 1

The example below demonstrates a weather information tool implementation. When a user asks about weather conditions, the model will:

1. Recognize the need for weather data
2. Extract location/unit parameters through structured tool definition
3. Return a formatted response using our `get_current_weather` tool

Expected result: The model should identify San Francisco as the location and celsius as the preferred unit, returning a structured tool response while maintaining full trace visibility in Langfuse.

In [ ]:
@observe(name="Tool Use Example")
def tool_use_api(query: str) -> list:
    langfuse_context.update_current_trace(
        user_id="nova-user-1",
        session_id="tool-use-session",
        tags=["lab1", "tool-use"],
    )

    messages = [{"role": "user", "content": query}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            },
        }
    ]

    return {
        "response": converse_tool_use(
            messages, tools, tool_choice="auto", **MODEL_CONFIG["nova_pro"]
        ),
        "statusCode": 200,
    }


print(tool_use_api(query="What's the weather like in San Francisco?, in celsius?"))

langfuse_context.flush()

![langfuse-traces-tool-use](./images/langfuse-trace-tool-use.png)

### Use Case 2
The example below demonstrates a multi-modal document transcription tool implementation. When a user asks to transcribe a document, the model will:

1. Recognize the schema for invoice transcription
2. Extract structured data from images through structured tool definition
3. Apply dependentSchemas to provide additional classification and reasoning for the extraction

Expected result: The model should extract all the metadata and line items from the invoice and output the structured data in JSON format with classification and reasoning for each field.

In [ ]:
system_prompt = """
<instructions>
  - Ensure to escape quotes in the JSON response
  - Return "" for missing field values
  - Apply dependentSchemas to all <document/> fields
</instructions>

<document>
{
    "$schema": "https://json-schema.org/draft/2020-12/schema",
    "$id": "/schemas/document",
    "type": "object",
    "description": "A document with the fields to transcribe",
    "properties": {
        "doc_type": { "properties":{"value":{"type":"string"}}, "description": "Type of Document: Receipt" },
        "receipt_number": { "properties":{"value":{"type":"string"}}, "description": "The receipt number or other identifier number" },
        "doc_amount_total": { "properties":{"value":{"type":"number"}}, "description": "The total receipt amount" },
        "currency": { "properties":{"value":{"type":"string"}}, "description": "AUD/USD/CAD" },
        "vendor_business_number": { "properties":{"value":{"type":"string"}}, "description": "Vendor's business identification number e.g. ABN" },
        "vendor_name": { "properties":{"value":{"type":"string"}}, "description": "Business name issueing the receipt" },
        "vendor_address": { "properties":{"value":{"type":"string"}}, "description": "Vendor's site address" },
        "vendor_phone": { "properties":{"value":{"type":"string"}}, "description": "Vendor's phone number" },
        "payment_method": { "properties":{"value":{"type":"string"}}, "description": "The payment type, e.g. EFTPOS, Card" },
        "date_issued": { "properties":{"value":{"format": "YYYY-MM-DDThh:mm:ss"}}, "description": "Date document was issued"},
        "line_items_amount_total": { "properties":{"value":{"type":"number"}}, "description": "Calculated sum of line item's line_amount fields" }
    },
    "dependentSchemas": {
        "value": {
            "properties": {
                "inference": { "type": "integer", "description": "0=EXPLICIT|1=DERIVED|2=MISSING|3=OTHER" },
                "source": { "type": "string", "description": "Source locations in the document for explicit and derived fields" }
            }
        }
    }
}
<document/>
"""


@observe(name="Vision Tool Use Example")
def vision_tool_use_api(
    query: str,
    image_url: str,
) -> list:
    langfuse_context.update_current_trace(
        user_id="nova-user-1",
        session_id="tool-use-session",
        tags=["lab1", "tool-use"],
    )

    messages = [
        {
            "role": "system",
            "content": system_prompt,
        },
        {
            "role": "user",
            "content": [
                {"type": "text", "text": query},
                {"type": "image_url", "image_url": {"url": image_url}},
            ],
        },
    ]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "transcribe_documents",
                "description": "Extract all <document/> fields with the highest accuracy following <instructions/>",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "documents": {
                            "type": "array",
                            "items": {"$ref": "/schemas/document"},
                        },
                    },
                    "required": ["documents"],
                },
            },
        }
    ]

    return {
        "response": converse_tool_use(
            messages, tools, tool_choice="auto", **MODEL_CONFIG["nova_pro"]
        ),
        "statusCode": 200,
    }


# image source: https://aws.amazon.com/blogs/machine-learning/announcing-expanded-support-for-extracting-data-from-invoices-and-receipts-using-amazon-textract/
print(
    vision_tool_use_api(
        query="Transcribe the invoice. Make sure to apply dependentSchemas to all <document/> fields",
        image_url="https://d2908q01vomqb2.cloudfront.net/f1f836cb4ea6efb2a0b1b99f41ad8b103eff4b59/2021/07/22/ml3911-img17.jpg",
    )
)

langfuse_context.flush()

# <img src="./images/langfuse-trace-tool-use-vision.png" width="800"/>

## Prompt Management
### What is prompt management?

Prompt management is a systematic approach to storing, versioning and retrieving prompts in LLM applications. Key aspects of prompt management include version control, decoupling prompts from code, monitoring, logging and optimizing prompts as well as integrating prompts with the rest of your application and tool stack.

Use Langfuse to effectively **manage** and **version** your prompts. Langfuse prompt management is a Prompt **CMS** (Content Management System).


### Why use prompt management?

Typical benefits of using a CMS apply here:

- Decoupling: deploy new prompts without redeploying your application.
- Non-technical users can create and update prompts via Langfuse Console.
- Quickly rollback to a previous version of a prompt.
- Compare different prompt versions side-by-side.

Platform benefits:

- Track performance of prompt versions in Langfuse Tracing.
- Performance benefits compared to other implementations:

-  No latency impact after first use of a prompt due to client-side caching and asynchronous cache refreshing.
-  Support for text and chat prompts.
-  Edit/manage via UI, SDKs, or API.


There are several ways you can create prompts in Langfuse:

-  Langfuse Console
-  Langfuse SDK
-  Langfuse API

In this workshop, we will be using Langfuse Python low-level SDK to create prompts by reusing the prompt exampels from the Modul1 - Prompt Engineering with Amazon Bedrock and Nova Model.


In [ ]:
# Initialize Langfuse client
langfuse = Langfuse()

# Create a chat prompt without COT
langfuse.create_prompt(
    name="software-development-project-management-without-COT",
    type="chat",
    prompt=[
        {
            "role": "user",
            "content": "You are a project manager for a small software development team tasked with launching a new app feature. You want to streamline the development process and ensure timely delivery.",
        }
    ],
    labels=["dev"],
    config={
        "model": MODEL_CONFIG["nova_pro"]["model_id"],
        "maxTokens": MODEL_CONFIG["nova_pro"]["inferenceConfig"]["maxTokens"],
        "temperature": MODEL_CONFIG["nova_pro"]["inferenceConfig"]["temperature"],
    },  # for Dev and experiment phase
)

In [ ]:
# Create a chat prompt with COT
langfuse.create_prompt(
    name="software-development-project-management-with-COT",
    type="chat",
    prompt=[
        {
            "role": "user",
            "content": """You are a project manager for a small software development team tasked with launching a new app feature. You want to streamline the development process and ensure timely delivery. Please follow these steps:\n
       {{step1}}\n
       \n
       {{step2}}\n
       \n
       {{step3}}\n
       \n
       {{step4}}\n""",
        }
    ],
    labels=["dev"],
    config={
        "model": MODEL_CONFIG["nova_pro"]["model_id"],
        "maxTokens": MODEL_CONFIG["nova_pro"]["inferenceConfig"]["maxTokens"],
        "temperature": MODEL_CONFIG["nova_pro"]["inferenceConfig"]["temperature"],
    },  # for Dev and experiment phase
)

You can see the two langfuse prompts are created successfully.

![langfuse-traces-prompt-management](./images/langfuse-prompt-management.png)


In [18]:
# Now, fetch both prompts and fill in the values for the variables and call the prompts
langfuse = Langfuse()

# Get current latest version of a prompt
sdpm_with_cot_prompt = langfuse.get_prompt(
    "software-development-project-management-with-COT", type="chat", label="dev"
)
# Insert variables into prompt template
sdpm_with_cot_prompt_compiled = sdpm_with_cot_prompt.compile(
    step1="Define Requirements",
    step2="Breakdown into Tasks",
    step3="Set Deadlines",
    step4="Monitor Progress and Optimize",
)

sdpm_without_cot_prompt = langfuse.get_prompt(
    "software-development-project-management-without-COT", type="chat", label="dev"
)
sdpm_without_cot_prompt_compiled = sdpm_without_cot_prompt.compile()

In [ ]:
sdpm_with_cot_prompt_compiled

Now you can add the prompt object to the generation call in the SDKs to link the generation in Langfuse Tracing to the prompt version. This linkage enables tracking of metrics by prompt version and name

In [20]:
# Converesation according to AWS spec including prompting + history
@observe()
def main():
    langfuse_context.update_current_trace(
        name="prompt-management-trace",
        user_id="nova-user-1",
        session_id="link-prompt-session",
        tags=["lab1"],
    )

    messages = [
        {
            "role": sdpm_with_cot_prompt_compiled[0]["role"],
            "content": sdpm_with_cot_prompt_compiled[0]["content"],
        }
    ]

    simple_chat(
        model_config=MODEL_CONFIG["nova_pro"],
        messages=messages,
        prompt=sdpm_with_cot_prompt,
    )

    messages = [
        {
            "role": sdpm_without_cot_prompt_compiled[0]["role"],
            "content": sdpm_without_cot_prompt_compiled[0]["content"],
        }
    ]

    simple_chat(
        model_config=MODEL_CONFIG["nova_pro"],
        messages=messages,
        prompt=sdpm_without_cot_prompt,
    )


main()
langfuse_context.flush()

You can see the trace is linked to the prompt version and the prompt name. 

![langfuse-traces-prompt-management](./images/langfuse-link-prompt.png)

## Lab1 Summary:
In Lab1, we explored the basics of integrating Langfuse with AWS to manage prompt traces effectively.
We demonstrated how to update traces, link prompts to user sessions, and visualize these linkages with a practical example.

As we conclude this lab, take a moment to reflect on the foundational skills you've gained.
Now, if you are at an AWS event, you can return to the workshop studio for additional instructions before moving into the next lab, where we will dive deeper into RAG related tracing and evaluation.
